In [1]:
import os
import configparser
import argparse
import numpy as np
import signal
import progressbar
import tensorflow as tf

In [2]:
from auto_pose.ae import ae_factory as factory
from auto_pose.ae import utils as u

In [4]:
# main()
workspace_path = os.environ.get('AE_WORKSPACE_PATH')

experiment_name = "my_autoencoder"
experiment_group = "exp_group"

at_step = None

cfg_file_path = u.get_config_file_path(workspace_path, experiment_name, experiment_group)
log_dir = u.get_log_dir(workspace_path, experiment_name, experiment_group)
checkpoint_file = u.get_checkpoint_basefilename(log_dir)
ckpt_dir = u.get_checkpoint_dir(log_dir)
dataset_path = u.get_dataset_path(workspace_path)
print("cfg_file_path", cfg_file_path)
print("log_dir", log_dir)
print("checkpoint_file", checkpoint_file)
print("ckpt_dir", ckpt_dir)
print("dataset_path", dataset_path)

('cfg_file_path', '/home/yq/Data/23_pose_estimation/aae/autoencoder_ws/cfg/exp_group/my_autoencoder.cfg')
('log_dir', '/home/yq/Data/23_pose_estimation/aae/autoencoder_ws/experiments/exp_group/my_autoencoder')
('checkpoint_file', '/home/yq/Data/23_pose_estimation/aae/autoencoder_ws/experiments/exp_group/my_autoencoder/checkpoints/chkpt')
('ckpt_dir', '/home/yq/Data/23_pose_estimation/aae/autoencoder_ws/experiments/exp_group/my_autoencoder/checkpoints')
('dataset_path', '/home/yq/Data/23_pose_estimation/aae/autoencoder_ws/tmp_datasets')


In [5]:
args = configparser.ConfigParser()
args.read(cfg_file_path)

['/home/yq/Data/23_pose_estimation/aae/autoencoder_ws/cfg/exp_group/my_autoencoder.cfg']

In [7]:
with tf.variable_scope(experiment_name):
    dataset = factory.build_dataset(dataset_path, args)
    queue = factory.build_queue(dataset, args)
    encoder = factory.build_encoder(queue.x, args)
    decoder = factory.build_decoder(queue.y, encoder, args)
    ae = factory.build_ae(encoder, decoder, args)
    codebook = factory.build_codebook(encoder, dataset, args)
    saver = tf.train.Saver(save_relative_paths=True)

batch_size = args.getint('Training', 'BATCH_SIZE')
model = args.get('Dataset', 'MODEL')

gpu_options = tf.GPUOptions(per_process_gpus_memory_fraction=0.7)
config = tf.ConfigProto(gpu_options=gpu_options)


ValueError: Variable my_autoencoder/conv2d/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/yq/anaconda3/envs/aae/lib/python2.7/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/home/yq/anaconda3/envs/aae/lib/python2.7/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/home/yq/anaconda3/envs/aae/lib/python2.7/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/home/yq/anaconda3/envs/aae/lib/python2.7/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/home/yq/anaconda3/envs/aae/lib/python2.7/site-packages/tensorflow_core/python/framework/op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)


In [ ]:
with tf.Session(config=config) as sess:
    print(ckpt_dir)
    print('#'*20)
    
    factory.restore_checkpoint(sess, saver, ckpt_dir, at_step=at_step)
    
    
    if model=='dsprites':
        codebook.update_embedding_dsprites(sess, args)
    else:
        codebook.update_embedding(sess, batch_size)
    
    print('Saving new checkpoint ..')
    
    saver.save(sess, checkpoint_file, global_step=ae.global_step)
    
    print('done')